# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

df = pd.read_csv('/datasets/users_behavior.csv')
display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Изучим данные. Для целевого признака выбираем столбец с названием тарифа, остальной дата сет для обучающих признаков.

## Разбейте данные на выборки

In [2]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features, features_test, target, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345, stratify=target)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.3, random_state=12345, stratify=target)

display(features.shape)
display(features_test.shape)
display(target.shape)
display(target_test.shape)
display(features_valid.shape)
display(target_valid.shape)


(2410, 4)

(804, 4)

(2410,)

(804,)

(723, 4)

(723,)

Как я понял, таким образом лучше вообще не поступать и лучше использовать gridsearch, но по заданию делим на 3 выборки. Сначала датасет делим на тестовую и тренировочную выборки 3:1, далее тренировочную разбиваем в таком же соотношении на тренировочную и валидационную (0.3 что бы валидационная выборка составляла примерно 1\4 от всего датасета).

## Исследуйте модели

### Начнем исследование моделей с  обучающего дерева

In [3]:
best_model = None
best_result = 0

for i in range(1,20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model
        best_result = model.score(features_valid, target_valid) 
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print('Лучшая модель:', best_model)

Accuracy наилучшей модели на валидационной выборке: 0.8063623789764869
Лучшая модель: DecisionTreeClassifier(max_depth=8, random_state=12345)


В цикле обучаем модель на тренировочной выборке, проверяем результат на валидационной, получаем неплохой результат.

### Исследование модели: случайный лес

In [4]:
%%time
best_model_forest = None
best_result_forest = 0
for est in range(1, 25):
    for depth in range(1,10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid) 
        if result > best_result_forest:
            best_model_forest = model 
            best_result_forest = model.score(features_valid, target_valid)  

print("Accuracy наилучшей модели на валидационной выборке:", best_result_forest, best_model_forest)

Accuracy наилучшей модели на валидационной выборке: 0.8105117565698479 RandomForestClassifier(max_depth=8, n_estimators=22, random_state=12345)
CPU times: user 6.36 s, sys: 36.3 ms, total: 6.4 s
Wall time: 6.41 s


Во вложенном цикле обучаем лес, перебирая 2 гиперпараметра est & depth, получаем так же неплохой результат. 

### Исследование модели: логистическая регрессия

In [5]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
model.predict(features_valid)
print(model.score(features_valid, target_valid))

0.7496542185338866


Обучаем модель на тренировочных данных, проверяем на валидационных - получаем быструю, но некачественную модель.
Соответственно для дальнейшей работы с тестовыми данными выбираем наиболее результативную модель обучающего леса.

## Проверьте модель на тестовой выборке

In [6]:
best_model_forest_test = None
best_result_forest_test = 0
for est in range(1, 25):
    for depth in range(1,10):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        result = model.score(features_test, target_test) 
        if result > best_result_forest_test:
            best_model_forest_test = model 
            best_result_forest_test = model.score(features_test, target_test)  

print("Accuracy наилучшей модели на тестовой выборке:", best_result_forest_test, best_model_forest_test)

Accuracy наилучшей модели на тестовой выборке: 0.8171641791044776 RandomForestClassifier(max_depth=8, n_estimators=11, random_state=12345)


In [7]:
test_result = best_model_forest.score(features_test, target_test)
print(test_result)

0.8121890547263682


## (бонус) Проверьте модели на адекватность

Попробуем проверить на адекватность путем сравнения результатов работы модели со случайностью.

In [8]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

данные распределены примерно 2 к 1 в пользу тарифа смарт, то есть при случайном распределении класса вероятность допустить ошибку будет около 30%, что приведет к точности примерно в 70%. 


Как вариант можно использовать Dummy-модель, которая всегда предсказывает наиболее часто встречающийся класс, результат тот же.

In [9]:
model_bl = DummyClassifier()
model_bl.fit(features_train, target_train)
score = model_bl.score(features_test, target_test)
print(f"The baseline accurancy: {score:.2%}")

The baseline accurancy: 69.40%
